<center> <h1> Simulación con dinámica molecular de un gas con potencial Lennard-Jones </h1> </center>

## Disclaimer

Parte del código ha sido creado empleando la ayuda de diferentes asistentes de IA como GitHub Copylot, ChatGPT o DeepSeek si bien el análisis ha sido completamente genuino.

## Introducción

Un potencial de tipo Lennard-Jones se usa para caracterizar sistemas en los que la interacción entre partículas es atractiva a distancias largas y altamente repulsiva cuando esta se encuentran muy cerca. El uso de este tipo de potencial para simular sistemas de partículas nos es útil para poder crear modelos de sistemas de partículas con interacciones microscópicas como átomos o moléculas sin que sea muy costoso a nivel computacional.

El potencial de Lennard-Jones viene descrito mediante la siguiente ecuación
$$
V(r) = 4 \epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right]
$$

Donde $\epsilon$ determina la profundidad del potencial y $\sigma$ está relacionada con la posición del mínimo. La primera imagen muestra un potencial para escala de energía genérica $\epsilon$ y distancia de interacción genérica $\sigma$. Cuando se disponen las partículas del sistema de forma organizada y separada entre sí una distancia $r = 2 ^(1/6) \sim 1.22\sigma$ estas caen en un pozo de potencial que en primera aproximación y para desplazamientos pequeños es armónico, como se muestra en la segunda imagen

<div style="display: flex;">
<img src="Animaciones y graficas/Potencial.png" width="500" style="margin-right: 20px">
<img src="Animaciones y graficas/Potencial simetrico.png" width="500">
</div>


En la primera imagen la asíntota en $ r = 0 $  simula un núcleo duro o una fuerza fuerte que repele entre sí a las partículas cuando se acercan demasiado haciendo que reboten entre sí. 



<div style="display: flex;">
<img src="Animaciones y graficas/Condiciones_periodicas.PNG" width=400 style="margin-right: 12px;">

Este proyecto consiste en la simulación de un conjunto de partículas interaccionando entre sí mediante un potencial de tipo Lennard-Jones encerradas en una caja y sometidas a condiciones de contorno periódicas aplicando condición de imagen mínima para la posición de las partículas.

Cuando una partícula atraviesa una de las paredes aparece en la pared opuesta con la misma dirección y sentido en la velocidad que seguiría tras atravesar la pared. Además al calcular las distancias entre partículas se asume que detrás de cada pared hay una copia exacta de todas las partículas en la caja, por lo que si dos partículas se encuentran a una distacia mayor a $ L/2 $ se le resta $ L/2 $ para poder representar una red o conjunto virtualmente infinito de partículas trabajando con tan solo unas pocas.

 Se empleará un algoritmo de Verlet de un solo paso (Verlet-Velocity) para hacer el cálculo basado en el cálculo de la posición de cada partícula en cada paso calculando la aceleración que sufre cada partícula debida a la interacción con el resto
</div>

## Desarrollo del programa

### Gestión del proyecto

La simulación se ha llevado a cabo en C++17 y el análisis de datos en Python 3.12.8 . Todos los programas, animaciones y gráficas se puedne encontrar en el repositorio de GitHub en la carpeta designada
- Compu/
    - Lennard_Jones/
        - Analisis/
        - Animaciones y graficas/
        - Simulacion/
            - Datos/
            - lennard-jones-lib/
            - ...


### Comando de compilación

```bash
g++ -I./lennard-jones-lib/include -L./lennard-jones-lib/build -fopenmp Lennard_Jones_sin_paralelo.cpp -llennardjones -o simulacion.exe
./simulacion.exe

## Estabilidad de sistema

Para sistemas governados por fuerzas centrales, como el caso de este sometido a un potencial Lennard-Jones, las fuerzas que actúan sobre las partículas son consrvativas, por lo que se cumple la ley de conservación de la energía mecánica

$$
E = T + V = cte
$$

Una simulación en la que se observa una variación con el tiempo de la energía del sistema es señal de errores en la programación, por lo que se comprobará la estabilidad del sistema y la conservación de la energía mecánica, para ello se ha planteado la siguiente simulación.

- Número de partículas del sistema: $N = 100$
- Paso temporal entre dos pasos del algoritmo de evolución: $h = 0.02$
- Ancho de la caja: $L = 10$
- Módulo de las velocidades iniciales de las partículas con ángulo aleatorio: $v = 10$

El siguiente video muestra la simulación a la izquierda y la evolución de las energís cinética, potencial y mecánica, donde se observa que en efecto la energía mecánica se conserva.

Las fluctuaciones en la energía total son tan pequeñas que seguramente estén debidas a términos de redondeo.

<img src="Animaciones y graficas/Animacion_100_5000_10.gif">

## Análisis estadístico de la velocidad

### Inicio con velocidades aleatorias

Para un sistema de partículas clásicas que interaccionan entre sí a corta distancia intercambiando momento y energía mediante choques elásticos se puede demostrar que si las velocidades sobre los ejes de las partículas vienen dadas por una distribución normal, $N(\mu, \sigma)$, la el módulo de la velocidad de las partículas del sistema sigue una distribución del tipo Maxwell-Boltzmann de dimensión $d$, la dimensión del sistema, 2D en este caso.

Para comprobar si el potencial Lennard-Jones genera un sistema en el que se cumpla la ecuación de Boltzmann, y por tanto que evolucione a un estado en el que las velocidades siguen una distribución Maxwell-Boltzmann se ha inicializado el sistema con las velocidades de las partículas configuradas de la siguiente forma.

- Hay $N$ partículas
- Las partículas tienen masa $m = 1$
- Tomamos $k_B = 1$ para simplificar los cálculos
- Se selecciona un valor para $|v|$ idéntico para todas las partículas
- Se selecciona la dirección de la velocidad mediante una distribución uniforme entre $0$ y $2\pi$

Sin embargo por el hecho de que $\theta$ venga dada por una distribución uniforme entre $0$ y $2\pi$, $v_x = v \cos(\theta)$ y $v_y = v \sin(\theta)$, se observa que las velocidades sobre los ejes, $v_x$ y $v_y$, no siguen una distrubución normal, sino una distribución arcoseno generalizada.

<img src="Animaciones%20y%20graficas/PDF%20arcoseno.png">

Por lo que el estudio de unas pocas partículas no mostraría el resultado esperado, sin embargo empleando el teorema central del límite si tomamos como datos las velocidades de las partículas en todos los instantes en un intervalo de tiempo, cuando el sistema ya ha alcanzado el *equilibrio*, se observa como las componentes de los ejes de la velocidad convergen a una distribución normal de media $\mu = 0$ y varianza $\sigma^2 = T k_B/m$

$$
P(v_x) = \sqrt{\frac{m}{2\pi k_BT}} \; e^{-\frac{m{v_x}^2}{2k_BT}}
$$

Haciendo uso de herramientas de estadística se demuestra que al distribución de probabilidad que sigue el módulo de la velocidad es Maxwell-Boltzmann bidimensional dada por el parámetro $a = T k_B/m$.

$$
P(v) = \left(\frac{m}{k_BT}\right)ve^{-\frac{mv^2}{2k_BT}}
$$

Los parámetros estadísticos de estas distribuciones vienen dados por la temperatura del sistema, la cual se puede calcular mediante la teoría cinética de los gases como

$$
k_B T = \frac{\langle E_{cinetica} \rangle}{N} = \frac{1}{2} m \langle v^2 \rangle \Rightarrow T = \frac{m}{2k_B} \langle v^2 \rangle
$$

La siguiente figura muestra un histograma de la distribución de probabilidad de las velocidades entre los tiempos $t = 20-50$ donde se observa claramente que se ajusta a la distribución teórica, tanto para $v$ como para $v_x$. Se ha seleccionado $|v| = 10$ para observar la animación claramente.

<img src="Animaciones%20y%20graficas/Distribuciones%20de%20probabilidad%2025000%20pasos%20y%2016%20partículas.png">

La siguiente animación muestra la evolución en la distribución de las velocidades según el sistema alcanza el equilibrio. Inicialmente se observa claramente como la distribución es muy picuda en torno a $v = 10$ ya que ese es el valor inicial del módulo de la velocidad que se le ha dado a todas las partículas, pero al avanzar el sistema hacia el equilibro la distribución de velocidades tiende a ajustarse a la distribución Maxwell-Boltzmann de temperatura dada por la teoría cinética de los gases. Se observan fluctuaciones en torno al valor teórico de la distribución, sin embargo aumentando el número de medidas, como se observa en la figura anterior, el ajuste es casi perfecto.


<img src="Animaciones y graficas/Animacion velocidades 100 partículas y 5000 pasos.gif">

### Comparación para diferentes temperaturas

Si modificamos la velocidad inicial de las partículas observamos que las distribuciones de la velocidad varían ya que la temperatura depende directamente del promedio de $v^2$, por lo que cuanto mayor sea la velocidad inicial más *achatada* será la distribución ya que aumentará la varianza de esta (que viene dada por la temperatura) que la determina, por lo que los valores de la velocidad estarán más repartidos, esto es, cuanto más frío el sistema más cercanas a cero estarán las velocidades y cuanto más caliente más repartidas en todo el rango posible estarán.

<img src="Animaciones%20y%20graficas/Histogramas de 4 velocidades y 16 partículas.png">

### Movimiento inicial de la caja

Ahora supongamos que se le da un impulso inicial al sistema en el eje x, esto implica que a las velocidades iniciales de las partículas se les sumará un valor adicional $v'$ en el eje x, por lo que la distribución de velocidades en este eje se desplazará alcanzando la media en este valor $v'$. Mientras que el valor medio de la velocidad en este eje se desplaza el valor de la varianza es el mismo ya que, de una forma intuitiva, la temperatura del sistema no cambia ya que las medidas en este sistema son equivalentes a movernos nosotros respecto a la caja con velocidad $-v'$ y hacer el análisis, como la temperatura del sistema no varía al movernos nosotros la varianza de las distribuciones debe ser la misma que en el caso anterior. En la siguiente figura se muestran las distribuciones de la velocidad en el eje x para simulaciones con un impulso inicial de 0, 1, 2, 3 y 4, donde se observa claramente como la media de cada distribución se ha desplazado al valor del impulso inicial mientras que la varianza es la misma.

<img src="Animaciones%20y%20graficas/Distribuciones eje x 16 partículas diferentes impulsos.png">

## Ecuación de estado

El uso del potencial de Lennard-Jones para llevar a cabo grandes simulaciones de partículas nos es útil para reproducir fluidos por ordenador. Un ejemplo es el de un gas diluido en el que las partículas no interactúan con el medio excepto entre sí a corta distancia (respecto al tamaño de la caja), en este caso es conveniente emplear la aproximación de gases ideales para estudiar el medio. Este tipo de sistemas poseen una ecuación de estado sencilla

$$
PV=NRT
$$

Por lo que se va a estudiar la presión del sistema para diferentes configuraciones y así obtener la ecuación de estado. El cálculo de la presión se lleva a cabo aplicando la conservación del momento en los choques de las partículas sobre las paredes. En el código se define una variable `presion` y cada vez que una partícula atraviesa una pared aplicaremos la condición de conservación del momento, como si la partícula hubiese rebotado, sumándole el momento en esa dirección a la presión.

```mermaid
flowchart LR
    Z((momento_pared = 0)) -->A
    A[Comienzo del paso] --> B
    B[Atraviesa una pared] --> C
    C{Qué pared es}
    C --> |Horizontal| D
    C --> |Vertical| E
    D[momento_pared += 2py] --> F
    E[momento_pared += 2px] --> F
    F[Fin del paso] --> G
    G{¿Fin de la simulación?}
    G --> |Sí| H
    G --> |No| A
    H[fuerza = momento_pared/tiempo de simulacion
    presion = fuerza / 4L]

```

Para que coincida con las unidades de presión dividimos entre el tiempo de simulación para obtener unidades de fuerza y entre el perímetro de la caja para obtener presión. Se divide por el perímetro porque $P = \frac{F}{S}$ pero en un sistema bidimensional la superficie de la caja, o su frontera, es el borde, que es unidimensional. Para comprobar si el sistema obedece la ecuación de estado de los gases ideales realizaremos 900 simulaciones de 25000 pasos en las que variaremos independientemente

- $N$: el número de partículas en la simulación desde 20 hasta 100
- $v$: la velocidad inicial de las partículas y por lo tanto la temperatura desde 1 hasta 10
- $L$: el tamaño de la caja desde 10 hasta 100

Calculando la presión del sistema en cada caso y representando esta independientemente frente a estas tres magnitudes para comprobar la linealidad. Las siguientes figuras muestran gráficas de la presión del sistema frente a la temperatura, el volumen (bidimensional $L^2$) de la caja y el número de partículas. La temperatura es calculada mediante la ecuación empleada en el segundo apartado
$$
T = \frac{1}{2}\langle v^2 \rangle
$$
Asumiendo $k_B = 1$ y $m = 1$. 

El ajuste frente a $L$ se hace con los valores de $1/L^2$ ya que en la ecuación de estado de los gases ideales $P$ depende inversamente del volumen, que es la superficie de la caja en nuestro caso.

<img src="Animaciones%20y%20graficas/Presión frente a Temp para N = 10.png">

<img src="Animaciones%20y%20graficas/Presión frente a L para Temp = 17.937449.png">

<img src="Animaciones%20y%20graficas/Presión frente a N para L = 70.png">

No se muestran las gráficas para $L=10$ ya que el tamaño de la caja es tan pequeño que para un gran número de partículas ocurren colisiones entre ellas con velocidad tan alta que el acercamiento al núcleo hace que el sistema explote.

Para corroborar la linealidad de la presión frente a $T$, $N$ y $1/L^2$ se hace un ajuste de uno de los conjuntos de datos para cada uno de ellos.

- $P$ frente a $T$ para $L = 20$ y $N = 10$
- $P$ frente a $1/L^2$ para $N = 100$ y $T = 17.98$
- $P$ frente a $N$ para $L = 70$ y $T = 40.20$

<img src="Animaciones y graficas/Ajuste.png">

Cuyos coeficientes de Pearson son 

- 0.999965
- 0.999134
- 0.997925

Como podemos observar por las gráficas y los valores del coeficiente de Pearson muy cercanos a 1 los datos se relacionan de forma lineal, por lo que el sistema cumple la ecuación de estado de los gases ideales, al menos en las condiciones estudiadas.

## Transición sólido-líquido

Hasta ahora hemos estudiado como evoluciona el sistema cuando sus partículas se encuentran inicialmente en movimiento tratando a este como un gas, sin embargo es interesante llevar a cabo un estudio del sistema cuando inicialmente las partículas se encuentran en reposo ya que se puede obsevar como la estructura que adquiere es una configuración crsitalina típica de un sólido. Para ello vamos a inicializar el sistema con una estructura inicial en cuadrícula par observar como se adquiere una configuración final en el que cada partícula está rodeada de 3. El siguiente vídeo muestra como inicialmente las partículas, durante un intervalo de tiempo, permanecen en la estructura en cuadrícula, pero al tratarse esta de un estado de equilibrio inestable las pequeñas perturbaciones debidas a errores de redondeo en el programa generan un movimiento en las partículas capaz de generar una transición de una configuración cristalina inestable a una estable como es la configuración en triángulos.

<img src="Animaciones y graficas/Animacion_16_250000_4.gif">

Haciendo un estudio de la velocidad de las partículas en este sistema se observa que al igual que en el caso de los gases, en el equilibro las velocidades se distribuyen según Maxwell-Boltzmann, sin embargo para que el sistema alcance esta distribución debe alcanzar el equilibrio, y en el caso del sólido requiere más pasos, lo que se ve claramente en el siguiente video ya que los valores experimentales de la velocidad solo se ajustan a los teóricos cuando ha pasado mucho tiempo.

<img src="Animaciones y graficas/Animacion velocidades inicio quieto.mp4">

### Estudio de estabilidad del sólido

Para estudiar si existe un cambio de fase de esta estructura sólida a un estado líquido o gaseoso se va a calentar el sistema lentamente reescalando las velocidades por un factor 1.5 en los tiempos [20, 30, 35, 45] y observando cómo evoluciona el sistema. El siguiente video muestra como según aumenta la temperatura del sistema, este pierde la estructura sólida y tiende a formar patrones desordenados típicos de los fluidos

<img src="Animaciones y graficas/Animacion primer calentamiento.gif">

Para observar más claramente esta transición de fase se estudia la variación cuadrática media del sistema
$$
\Delta r(t)^2 = \langle(r(t) - r(0))^2\rangle
$$
la cual muestra cómo se desplaza una partícula respecto a su posición incial. Como podemos observar en la siguiente figura al aumentar la temperatura (pasos señalados en rojo) llega un momento en el que se pierde la estructura sólida ya que las fluctuaciones de la posición aumentan drásticamente y las partículas dejan de situarse en regiones centradas en su posición inicial distribuyéndose de forma aleatoria por el volumen

<img src="Animaciones y graficas/VarPosicion.png">

### Estudio de la fusión: temperatura crítica

A la hora de determinar la temperatura crítica del sistema, esto es, la temperatura a la que pasa de ser sólido a líquido, la temperatura del cambio de fase, se va a estudiar la separación entre dos partículas aleatorias del sistema con el tiempo.

$$
\Delta r_{ij}(t)^2 = \langle(r_i(t) - r_j(t))^2\rangle
$$

Y se irá calentando el sistema más lentamente para poder determinar la temperatura crítica con mayor precisión, reescalando un factor 1.1 las velocidades en los tiempos [30, 60, 90, 120, ...]. La siguiente figura muestra la evolución de la separación entre partículas con el tiempo según se aumenta la temperatura (señalado en rojo).

<img src="Animaciones y graficas/Separacion.png">

Se observa que en $t=210$ hay un cambio drástico en la separación entre partículas, correspondiendo al cambio de fase del sistema. La temperatura en ese intervalo es 1.618.

## Análisis del tiempo de ejecucion

Finalmente vamos a llevar a cabo un análisis del tiempo de ejecucion del programa teniendo en cuenta diversos factores

- Número de hilos en la paralelización
- Dispositivo (Joel, portátil, etc)
- Tipo de compilador (solo en Joel)

### Dependencia con la paralelización

Se ha paralelizado el código empleando la librería `<omp.h>` para `C++` para los bucles críticos. Como se el algoritmo de Verlet tiene una fuerte dependencia entre un paso y el siguiente los únicos bucles paralelizables son los internos dentro de un mismo paso como el cálculo de las aceleraciones o las posiciones. Debido a esto y como dichos bucles recorren el número de partículas, el aumento de rendimiento con la paralelización no es significativo para un número pequeño de estas

### Comparación de dispositivos

### Comparación de compiladores